In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
import pandas as pd
import seaborn as sns  # data visualization
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/Users/sse/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


../input/animal-crossing-new-horizons-nookplaza-dataset/villagers.csv
../input/acnh-villager-popularity/acnh_villager_data.csv


# Introduction
The goal of this project is to analyse the relationship between animal crossing new horizon villager popularity amongst the player base and certain villager attributes. 

We will be analysing the Gender,Personality, Species, and Style of a villager. 

# Data Initilization and Cleaning

In [2]:
vlgr_df = pd.read_csv("../input/animal-crossing-new-horizons-nookplaza-dataset/villagers.csv")
popul_df = pd.read_csv("../input/acnh-villager-popularity/acnh_villager_data.csv")

# -- STEFANOS -- Replicate Data

In [3]:
df = vlgr_df
intended_df_size_in_MB = 256
factor = intended_df_size_in_MB*(2**20)/(vlgr_df.memory_usage(index=True).sum()+popul_df.memory_usage(index=True).sum())
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
vlgr_df = df

df = popul_df
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
popul_df = df

popul_df.info()
vlgr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1749881 entries, 0 to 1749880
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   tier    int64 
 1   rank    int64 
 2   name    object
dtypes: int64(2), object(1)
memory usage: 40.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656667 entries, 0 to 1656666
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   Name             1656667 non-null  object
 1   Species          1656667 non-null  object
 2   Gender           1656667 non-null  object
 3   Personality      1656667 non-null  object
 4   Hobby            1656667 non-null  object
 5   Birthday         1656667 non-null  object
 6   Catchphrase      1656667 non-null  object
 7   Favorite Song    1656667 non-null  object
 8   Style 1          1656667 non-null  object
 9   Style 2          1656667 non-null  object
 10  Color 1          1656667 non-null  object
 11  Color 2 

In [4]:
vlgr_df.head()

,Name,Species,Gender,Personality,Hobby,Birthday,Catchphrase,Favorite Song,Style 1,Style 2,Color 1,Color 2,Wallpaper,Flooring,Furniture List,Filename,Unique Entry ID
0,Admiral,Bird,Male,Cranky,Nature,27-Jan,aye aye,Steep Hill,Cool,Cool,Black,Blue,dirt-clod wall,tatami,717;1849;7047;2736;787;5970;3449;3622;3802;410...,brd06,B3RyfNEqwGmcccRC3
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul,sidekick,Go K.K. Rider,Active,Simple,Blue,Black,concrete wall,colorful tile flooring,7845;7150;3468;4080;290;3971;3449;1708;4756;25...,squ05,SGMdki6dzpDZyXAw5
2,Agnes,Pig,Female,Big Sister,Play,21-Apr,snuffle,K.K. House,Simple,Elegant,Pink,White,gray molded-panel wall,arabesque flooring,4129;7236;7235;7802;896;3428;4027;7325;3958;71...,pig17,jzWCiDPm9MqtCfecP
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct,ayyyeee,Go K.K. Rider,Active,Active,Red,White,concrete wall,green rubber flooring,1452;4078;4013;833;4116;3697;7845;3307;3946;39...,gor08,LBifxETQJGEaLhBjC
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun,it'sa me,Forest Life,Simple,Simple,Red,Blue,yellow playroom wall,green honeycomb tile,4763;3205;3701;1557;3623;85;3208;3584;4761;121...,crd00,REpd8KxB8p9aGBRSE


In [5]:
popul_df.head()

,tier,rank,name
0,1,1,Raymond
1,1,2,Marshal
2,1,3,Shino
3,1,4,Sherb
4,1,5,Sasha


### 1. Checking for null 

In [6]:
vlgr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156400 entries, 0 to 156399
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Name             156400 non-null  object
 1   Species          156400 non-null  object
 2   Gender           156400 non-null  object
 3   Personality      156400 non-null  object
 4   Hobby            156400 non-null  object
 5   Birthday         156400 non-null  object
 6   Catchphrase      156400 non-null  object
 7   Favorite Song    156400 non-null  object
 8   Style 1          156400 non-null  object
 9   Style 2          156400 non-null  object
 10  Color 1          156400 non-null  object
 11  Color 2          156400 non-null  object
 12  Wallpaper        156400 non-null  object
 13  Flooring         156400 non-null  object
 14  Furniture List   156400 non-null  object
 15  Filename         156400 non-null  object
 16  Unique Entry ID  156400 non-null  object
dtypes: object(

In [7]:
popul_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165200 entries, 0 to 165199
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tier    165200 non-null  int64 
 1   rank    165200 non-null  int64 
 2   name    165200 non-null  object
dtypes: int64(2), object(1)
memory usage: 3.8+ MB


### 2. Checking for mismatched names

In [8]:
# There are some missing/non-matching names 
vlgr_df["Name"].isin(popul_df['name']).sum()

154400

In [9]:
# vlgr_df does not have these names...
mismatch_names = popul_df["name"][popul_df["name"].isin(vlgr_df["Name"]) == False]
mismatch_names

2                  Shino
4                  Sasha
5                   Ione
25            Cephalobot
57                Étoile
               ...      
165087    Crackle(Spork)
165110               Ace
165115              Toby
165128             Frett
165186             Rilla
Name: name, Length: 10800, dtype: object

In [10]:
# %%time
# Data set is small enough to pick out the same names
# Correcting names in popul_df to match vlgr_df
popul_df['name'] = popul_df['name'].replace(['OHare'],"O\'Hare")
popul_df['name'] = popul_df['name'].replace(['Buck(Brows)'],"Buck")
popul_df['name'] = popul_df['name'].replace(['Renee'],"Renée")
popul_df['name'] = popul_df['name'].replace(['WartJr'],"Wart Jr.")
popul_df['name'] = popul_df['name'].replace(['Crackle(Spork)'],"Spork")

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 35.6 ms


In [11]:
# Checking if All names match
vlgr_df["Name"].isin(popul_df['name']).sum()

156400

In [12]:
# drop villagers that are in popul_df but not in vlgr_df
popul_df = popul_df.drop(popul_df[popul_df["name"].isin(vlgr_df["Name"]) == False].index)

### 3. Merging the two Dataframes

In [13]:
# Now that both df have same length, we can set index as names and combine the 2 dfs
popul_df.set_index('name', drop=True, inplace=True)
vlgr_df.set_index('Name', drop=True, inplace=True)

In [14]:
combined_df = popul_df.merge(vlgr_df, left_index=True, right_index=True)

In [15]:
# drop irrelevent columns
combined_df.drop(columns=['Furniture List', 'Filename', 'Unique Entry ID', "Wallpaper", "Flooring", "Birthday", "Favorite Song"], inplace=True)

#### Adding a new row named overall_ranking so we may know a villager's general ranking outside of their tier

In [16]:
combined_df.sort_values(['tier', 'rank'], inplace=True)
combined_df['overall_ranking'] = np.arange(1, len(combined_df)+1)
combined_df.insert(2, 'overall_ranking', combined_df.pop('overall_ranking'))

#### Setting Baseline overall ranking mean to compare against

In [17]:
overall_mean = combined_df.overall_ranking.mean()
print(f'The overall_mean is {overall_mean}, this would serve as a baseline for to compare against popularity performance of our features.')

The overall_mean is 31280000.5, this would serve as a baseline for to compare against popularity performance of our features.


In [18]:
combined_df.columns

Index(['tier', 'rank', 'overall_ranking', 'Species', 'Gender', 'Personality',
       'Hobby', 'Catchphrase', 'Style 1', 'Style 2', 'Color 1', 'Color 2'],
      dtype='object')

# Exploratory Data Analysis
As a preface, a higher overall_ranking would mean performing worse on the popularity rankings.
### 1. Gender

In [19]:
combined_df['Gender'].value_counts()

Male      32640000
Female    29920000
Name: Gender, dtype: int64

In [20]:
## -- STEFANOS: Disable plotting
# combined_df.groupby('tier').Gender.value_counts().plot.barh()
## -- STEFANOS-DISABLE-FOR-MODIN: Actually, it seems we can't even run the last part with Modin, so
## we only leave the groupby.
# combined_df.groupby('tier').Gender.value_counts()
combined_df.groupby('tier')

tier  Gender
1     Female      960000
      Male        960000
2     Female     2240000
      Male       1600000
3     Female     2880000
      Male       1760000
4     Female     4800000
      Male       4000000
5     Female     9280000
      Male       8800000
6     Male      15520000
      Female     9760000
Name: Gender, dtype: int64

For gender, there seems to be a disproporationate amount of male villagers in the lowest tier(6th tier) than female villagers, compared to other tiers. Discounting Tier 6, The number of male and female villagers are fairly even, with Male villagers having a slight lead in all tiers(except tier 6).

In [21]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(5, 5))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Gender", y='overall_ranking', data=combined_df)

Female villagers generally perform better than Male villagers in terms of overall ranking. 

In [22]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Gender", aggfunc='count')

Gender,Female,Male
tier,,
1,960000,960000
2,2240000,1600000
3,2880000,1760000
4,4800000,4000000
5,9280000,8800000
6,9760000,15520000


### 2. Species

In [23]:
# creating value counts dataframe for each species type
species_ranking = combined_df.groupby('Species').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')
species_ranking

/tmp/ipykernel_416040/3463245137.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  species_ranking = combined_df.groupby('Species').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')


,Species,overall_ranking
25,Octopus,2.693334e+06
9,Deer,8.288000e+06
34,Wolf,1.154182e+07
5,Cat,1.504348e+07
21,Koala,2.137778e+07
10,Dog,2.228000e+07
8,Cub,2.283000e+07
17,Hamster,2.348000e+07
32,Squirrel,2.369778e+07
30,Rhino,2.477333e+07


In [24]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(30,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Species', y="overall_ranking", data=species_ranking,label='mean overall-ranking', s=300)

Octopus, deer, wolves, cats and Koalas are most likely to be popular; while Kangaroos, Hippos, Mouse Pigs and Gorillas are the least likely to be popular. 

In [25]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(30, 10))
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x="Species", y='overall_ranking', hue='tier', s=100, data=combined_df)

Although Octopuses seem to be ranking highly in part due to the low amount of Octopuses amongst the villagers. 
Interesting trend can be observed, there exists a ranking cap for low ranking speices, for example, none of the Gorilla villagers have a ranking lower than 200, it is heavily skewed, and not normally distributed.  Indicating a clear non-preference for certain species by the playerbase. 

### 3. Personality

In [26]:
combined_df.Personality.value_counts()

Lazy          9600000
Normal        9440000
Snooty        8800000
Jock          8800000
Cranky        8800000
Peppy         7840000
Smug          5440000
Big Sister    3840000
Name: Personality, dtype: int64

In [27]:
# creating value counts dataframe for each personality type
personality_ranking = combined_df.groupby('Personality').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')

/tmp/ipykernel_416040/2251499037.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  personality_ranking = combined_df.groupby('Personality').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')


In [28]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(20,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Personality', y="overall_ranking", data=personality_ranking,label='mean personality ranking', s=300)

The playerbase seems to have a preference for Big sister, Normal, Peppy and sometimes Lazy type villagers.
While they dislike Cranky, Jock and Snooty villagers. 

In [29]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(10, 10))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Personality", y='overall_ranking', data=combined_df)

There seems to be a clear preference for Big Sister, Peppy and Normal Personality villagers, they have means below overall mean. Rankings are fairly normally distributed except for Smug villagers. On the other hand, Cranky and Snooty both have a mean clearly above the overall mean.

In [30]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Personality", aggfunc='count')

Personality,Big Sister,Cranky,Jock,Lazy,Normal,Peppy,Smug,Snooty
tier,,,,,,,,
1,NaN,NaN,NaN,640000.0,640000.0,NaN,320000.0,320000.0
2,160000.0,320000.0,320000.0,640000.0,960000.0,960000.0,320000.0,160000.0
3,800000.0,640000.0,480000.0,320000.0,1280000.0,640000.0,320000.0,160000.0
4,960000.0,800000.0,800000.0,1440000.0,1280000.0,1600000.0,960000.0,960000.0
5,1280000.0,2080000.0,2880000.0,3040000.0,3040000.0,2560000.0,800000.0,2400000.0
6,640000.0,4960000.0,4320000.0,3520000.0,2240000.0,2080000.0,2720000.0,4800000.0


### 4. Style

In [31]:
# generating value counts dataframe for each style type
style_ranking1 = combined_df.groupby('Style 1').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')
style_ranking2 = combined_df.groupby('Style 2').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')

/tmp/ipykernel_416040/3035286731.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  style_ranking1 = combined_df.groupby('Style 1').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')
/tmp/ipykernel_416040/3035286731.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  style_ranking2 = combined_df.groupby('Style 2').mean()['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')


In [32]:
# combining the 2 style columns and finding a mean
style_ranking = style_ranking1.copy()
style_series = (style_ranking1['overall_ranking'] + style_ranking2['overall_ranking'])/2
style_ranking["overall_ranking"] = style_series

In [33]:
style_ranking

,Style 1,overall_ranking
2,Cute,2.200213e+07
5,Simple,2.976031e+07
3,Elegant,3.350778e+07
1,Cool,3.460626e+07
4,Gorgeous,3.504125e+07
0,Active,3.610720e+07


In [34]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(20,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Style 1', y="overall_ranking", data=style_ranking, s=300)

A very clear preference for Cute styled villagers. Simple Styled Villagers have a ranking mean just about equal to the overall mean, while other style villagers have a slightly above overall mean mean. 

In [35]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(7, 7))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Style 1", y='overall_ranking', data=combined_df)
# plt.title('Style 1')
# plt.figure(figsize=(7, 7))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Style 2", y='overall_ranking', data=combined_df)
# plt.title('Style 2')

The clear preference is Cute style dressing, in both Style columns. In particular, in Style 2 column Cute Styled Villagers have a higher concetration in lower rankings. Other styles seem to have a fairly normally distributed ranking, with the exception of Active Style Villagers in Style 1, right skewed, but the ranking mean is significantly above the overall ranking mean.  

In [36]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Style 1", aggfunc='count')

Style 1,Active,Cool,Cute,Elegant,Gorgeous,Simple
tier,,,,,,
1,NaN,NaN,480000.0,320000.0,160000.0,960000.0
2,320000.0,320000.0,1280000.0,160000.0,320000.0,1440000.0
3,480000.0,1120000.0,1280000.0,800000.0,NaN,960000.0
4,320000.0,1760000.0,1760000.0,1120000.0,960000.0,2880000.0
5,2400000.0,2720000.0,2240000.0,2720000.0,1760000.0,6240000.0
6,4480000.0,4960000.0,3040000.0,3520000.0,2880000.0,6400000.0


In [37]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Style 2", aggfunc='count')

Style 2,Active,Cool,Cute,Elegant,Gorgeous,Simple
tier,,,,,,
1,NaN,320000.0,1120000.0,160000.0,NaN,320000.0
2,320000.0,320000.0,960000.0,320000.0,160000.0,1760000.0
3,320000.0,480000.0,1600000.0,480000.0,320000.0,1440000.0
4,1280000.0,1120000.0,2080000.0,1280000.0,1280000.0,1760000.0
5,2080000.0,2400000.0,2880000.0,2400000.0,2880000.0,5440000.0
6,4000000.0,4800000.0,1280000.0,4320000.0,4000000.0,6880000.0


# Conclusion
We may come to the conclusion, that the following attributes contribute to a villager's popularity:
- Gender: Despite Female Villagers having in general better popularity, this is likely due to the overwheling prescence of male villagers in the lowest tier. Other than the lowest tier, Male villagers in general perform slightly better.
- Species: Octopus, Wolf, Deer and Cat villagers perform the best. 
- Personality: Big Sister, Normal and Peppy villagers are in general the most popular. 
- Style: Cute Style villagers are very clearly the most popular